# 1. 부산시 남구 주정차위반이력

In [39]:
import pandas as pd
from pandas import DataFrame, Series
from datetime import datetime
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import re

# 지도 시각화
import json
import geojson
import folium
from folium.plugins import MarkerCluster

# 오픈 api
import requests 
from urllib.parse import urlparse

#오류 관련
import warnings

# 격자별 매핑 관련
import pyproj
from fiona.crs import from_epsg
import geopandas as gpd

# 소수점 자리
pd.options.display.float_format = lambda x: f'{x:.10f}'

pd.set_option('display.max_rows', 5000)
pd.set_option('display.max_columns',5000)
# 오류 무시
warnings.filterwarnings('ignore')
# 글씨체
mpl.rc('font', family='Malgun Gothic')

In [3]:
# 1. '1.부산시남구_주정차위반이력.csv'
# 전체 152,682 건
df = pd.read_csv('1.부산시남구_주정차위반이력.csv',encoding ='utf-8') ## 파일 임포트

In [4]:
# 외부데이터 크롤링
CCTV설치현황=pd.read_html('https://www.bsnamgu.go.kr/index.namgu?menuCd=DOM_000000104001007004',encoding='utf-8')
CCTV설치현황 = CCTV설치현황[0]
CCTV설치현황.drop(['연번', '동별개소','설치년도(최초)','성능개선(최종)','지구별','유예시간', '점심시간', '단속시간'],axis=1,inplace=True)

In [5]:
CCTV설치현황

,행정동,CCTV장소명,지번주소
0,대연1동,대연동 대연사거리CCTV,대연1동 1738-5
1,대연1동,대연 부산은행건너편CCTV,대연1동 889-2
2,대연3동,대연동 용소로 22-1앞CCTV,대연3동 61-11
3,대연3동,대연동 용소로11앞CCTV,대연3동 56-7
4,대연3동,대연동 대연비치앞CCTV,대연3동 16-2
5,대연3동,대연동 부경대정문앞CCTV,대연3동 502-2
6,대연3동,대연경성대부경대역CCTV,대연3동 73-15
7,대연3동,대연동 대천사거리CCTV,대연3동 567-1
8,대연3동,대연동 대천사거리1CCTV,대연3동 567-1
9,대연3동,대연동 리마크빌뒤편CCTV,대연3동 54-5


## 1.단속장소 cd_loc 전처리 (1)

### ◉ 주소 검색 함수 생성(address & keyword) : 카카오맵 API 사용

In [ ]:
def address(address): # 지번주소를 좌표로 반환하는 함수
    url = 'https://dapi.kakao.com/v2/local/search/address.json?&query=' + address
    result = requests.get(urlparse(url).geturl(), headers={'Authorization': 'KakaoAK 52aff49765b27f3c0683f304b70b3c56'}).json()
    
    if result['documents']==[]:
        lon, lat = np.NaN, np.NaN
        return lon, lat
        
    else:
        lon=result['documents'][0]['x'] #경도
        lat=result['documents'][0]['y'] #위도
        return lon, lat

def keyword(keyword): # 건물명을 좌표로 반환하는 함수
    url = 'https://dapi.kakao.com/v2/local/search/keyword.json?&query=' + keyword
    result = requests.get(urlparse(url).geturl(), headers={'Authorization': 'KakaoAK 52aff49765b27f3c0683f304b70b3c56'}).json()
    
    if result['documents']==[]:
        lon, lat = np.NaN, np.NaN
        return lon, lat
        
    else:
        lon=result['documents'][0]['x'] #경도
        lat=result['documents'][0]['y'] #위도
        return lon, lat

In [7]:
# 1
# df = pd.read_csv('1.부산시남구_주정차위반이력.csv',encoding ='utf-8')
df = pd.read_csv('1.부산시남구_주정차위반이력.csv',encoding ='utf-8') # 파일 임포트
단속장소=df.groupby('cd_loc',as_index=False)['cd_date'].count().sort_values(by='cd_date',ascending=False)
단속장소.rename(columns={"cd_date":'count'},inplace=True)

In [8]:
단속장소 # 15800개

,cd_loc,count
8304,대연사거리하나투어CCTV,3000
3049,대연동 부경대정문앞CCTV,2635
1362,감만동 현대@사거리CCTV,2305
7749,대연동 파크푸르지오CCTV,2301
2065,대연경성대부경대역CCTV,2091
...,...,...
7185,대연동 진남로 아주문구 부근(사선),1
7186,대연동 진남로 아주문구 부근(사선/횡단보도),1
7187,대연동 진남로 아주문구 부근(인도),1
7188,대연동 진남로 아주문구 부근(인도/곡각/횡단보도),1


In [9]:
#2 괄호및 특수문자 제거
regex= r'\([^)]*\)|\@'#|\정문$|\정문\s$'
for i in range(len(단속장소.cd_loc)):
    단속장소.cd_loc[i]= re.sub(regex,"", 단속장소 .cd_loc[i])

In [10]:
#3
remove=['상가','부근','입구','앞','옆','CCTV','인근','후문','대로변'] # 인도,정문 제거하면 안됨.
for i in remove:
    단속장소.cd_loc= 단속장소.cd_loc.str.replace(i,'')

In [11]:
#4
# 맨마지막 공백제거
regex2='\s$|\s\s$|\s\s\s$'# 공백 한개 있거나 두개 있는 경우 제외
for i in range(len(단속장소.cd_loc)):
    단속장소.cd_loc[i]= re.sub(regex2,"", 단속장소 .cd_loc[i])

In [13]:
# 5 # 9308개로 줄어듦.
단속장소 = 단속장소.groupby('cd_loc',as_index=False)['count'].sum().sort_values(by='count',ascending=False)

In [15]:
# 6: 위경도 컬럼 추가
단속장소['lon']=0
단속장소['lat']=0

In [18]:
# 지번주소를 좌표로 반환 - address 함수로 매칭
for idx,val in 단속장소.cd_loc.iteritems():
     단속장소.lon[idx],단속장소.lat[idx]=address(val)
단속장소

,cd_loc,count,lon,lat
5018,대연사거리하나투어,3826,nan,nan
5690,문현동 문현교차로,3233,nan,nan
1639,대연동 부경대정문,3032,nan,nan
8998,용호동 현대비치,2909,nan,nan
1090,대연경성대부경대역,2847,nan,nan
...,...,...,...,...
4543,대연동 진남로36번길 비비큐,1,129.0913025274,35.1382274666
4540,대연동 진남로36번길 6-0,1,129.0915640979,35.1383661446
4528,대연동 진남로36번길 18,1,129.0915442341,35.1388588805
4525,대연동 진남로34번길 3,1,129.0917198473,35.1382664004


In [21]:
단속장소[단속장소.lon.isna()].info() # 3687개; address 함수로 못걸러낸 개수 # keyword로 못걸러낸;1835개

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1835 entries, 5017 to 9307
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   cd_loc  1835 non-null   object 
 1   count   1835 non-null   int64  
 2   lon     0 non-null      float64
 3   lat     0 non-null      float64
dtypes: float64(2), int64(1), object(1)
memory usage: 71.7+ KB


In [20]:
# 건물명을 좌표로 반환하는 함수 keyword로 매핑
for idx,val in 단속장소[단속장소.lon.isna()].cd_loc.iteritems():
     단속장소.lon[idx],단속장소.lat[idx]=keyword(val)
단속장소

,cd_loc,count,lon,lat
5018,대연사거리하나투어,3826,129.0906255622,35.1352555052
5690,문현동 문현교차로,3233,129.0677799658,35.1369200688
1639,대연동 부경대정문,3032,129.1011880264,35.1333596545
8998,용호동 현대비치,2909,129.1084764384,35.1260931548
1090,대연경성대부경대역,2847,129.1005192438,35.1376003834
...,...,...,...,...
4543,대연동 진남로36번길 비비큐,1,129.0913025274,35.1382274666
4540,대연동 진남로36번길 6-0,1,129.0915640979,35.1383661446
4528,대연동 진남로36번길 18,1,129.0915442341,35.1388588805
4525,대연동 진남로34번길 3,1,129.0917198473,35.1382664004


In [22]:
# re_loc 컬럼 생성
단속장소['re_loc']=단속장소.cd_loc

In [23]:
# 컬럼 순서 변경
단속장소=단속장소[['cd_loc','re_loc','count', 'lon', 'lat']]

In [25]:
# keyword 이후 # 정규식
regex2=r'(\s\w*로|\w*로|\w*길|\s\w*길)'
for i in 단속장소.re_loc[단속장소.lon.isna()].index:
    단속장소.re_loc[i]=re.sub(regex2,"", 단속장소.re_loc[i])

In [26]:
# address 이후, keyword 이후 3번째 진행
for idx,val in 단속장소[단속장소.lon.isna()].re_loc.iteritems():
     단속장소.lon[idx],단속장소.lat[idx]=keyword(val)
단속장소

,cd_loc,re_loc,count,lon,lat
5018,대연사거리하나투어,대연사거리하나투어,3826,129.0906255622,35.1352555052
5690,문현동 문현교차로,문현동 문현교차로,3233,129.0677799658,35.1369200688
1639,대연동 부경대정문,대연동 부경대정문,3032,129.1011880264,35.1333596545
8998,용호동 현대비치,용호동 현대비치,2909,129.1084764384,35.1260931548
1090,대연경성대부경대역,대연경성대부경대역,2847,129.1005192438,35.1376003834
...,...,...,...,...,...
4543,대연동 진남로36번길 비비큐,대연동 진남로36번길 비비큐,1,129.0913025274,35.1382274666
4540,대연동 진남로36번길 6-0,대연동 진남로36번길 6-0,1,129.0915640979,35.1383661446
4528,대연동 진남로36번길 18,대연동 진남로36번길 18,1,129.0915442341,35.1388588805
4525,대연동 진남로34번길 3,대연동 진남로34번길 3,1,129.0917198473,35.1382664004


In [28]:
단속장소[단속장소.lon.isna()].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 862 entries, 5017 to 9307
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   cd_loc  862 non-null    object 
 1   re_loc  862 non-null    object 
 2   count   862 non-null    int64  
 3   lon     0 non-null      float64
 4   lat     0 non-null      float64
dtypes: float64(2), int64(1), object(2)
memory usage: 40.4+ KB


# 

## 2. CCTV설치현황(외부데이터 크롤링) 장소명 전처리

In [30]:
regex= r'\([^)]*\)|\@'#|\정문$|\정문\s$'
for i in range(len(CCTV설치현황)):
    CCTV설치현황.CCTV장소명[i]= re.sub(regex,"", CCTV설치현황.CCTV장소명[i])

In [31]:
remove=['상가','부근','입구','앞','옆','CCTV','정문','인근','뒤편','뒤','인도']
for i in remove:
    CCTV설치현황.CCTV장소명=CCTV설치현황.CCTV장소명.str.replace(i,'')

In [32]:
CCTV설치현황['str0']=CCTV설치현황.CCTV장소명.str.split('동',1).str[0]
CCTV설치현황['str1']=CCTV설치현황.CCTV장소명.str.split('동',1).str[1]

In [33]:
CCTV설치현황.str1[CCTV설치현황.str1.isna()]=CCTV설치현황['str0']

In [34]:
# CCTV설치현황 + 위경도좌표 
# 저장완료
#1 ; address
CCTV설치현황['lon']=np.nan
CCTV설치현황['lat']=np.nan
for idx,val in CCTV설치현황.지번주소.iteritems():
    CCTV설치현황.lon[idx],CCTV설치현황.lat[idx]=address(val)

In [35]:
# 2 ; keyword(도로명)
for idx, val in CCTV설치현황.CCTV장소명[CCTV설치현황.lon.isna()].iteritems():
    CCTV설치현황.lon[idx],CCTV설치현황.lat[idx]=keyword(val)

In [36]:
# 파일 저장 완료
CCTV설치현황.str1[1]='부산은행건너편'
CCTV설치현황.str1[19]='석포초교'
CCTV설치현황.str1[21]='남구청사거리'
CCTV설치현황.str1[28]='SK뷰'
CCTV설치현황.str1[32]='부산은행감만'
CCTV설치현황.str1[44]='베스티움'
CCTV설치현황.str1[45]='경동리인'

In [37]:
# 앞뒤 공백제거
regex3='^\s|^\s\s|\s$|\s\s$|\s\s\s$'
for idx in CCTV설치현황.str1.index:
    CCTV설치현황.str1[idx]=re.sub(regex3,'',CCTV설치현황.str1[idx])

## 3.매핑작업(1. 단속장소 & 2. CCTV설치현황)

In [40]:
# CCTV설치주소 & 단속장소 매핑
CCTV설치현황['단속장소인덱스']=CCTV설치현황.apply(lambda x: [], axis=1)
최종매칭전=단속장소[단속장소.lon.isna()] # 850여개

for idx,val in CCTV설치현황.str1.iteritems():
    #CCTV설치현황.단속장소인덱스[idx]=[]
    for i in 최종매칭전[최종매칭전.re_loc.str.contains(val)].index:
        CCTV설치현황.단속장소인덱스[idx].append(i)

In [41]:
# [] --> np.nan 처리
for idx,val in CCTV설치현황.단속장소인덱스.iteritems():
    if val==[]:
        CCTV설치현황.단속장소인덱스[idx]=np.nan

In [42]:
# 매핑작업
for idx,val in CCTV설치현황.단속장소인덱스[~CCTV설치현황.단속장소인덱스.isna()].iteritems():
    for j in val:
        단속장소.lon[j]=CCTV설치현황.lon[idx]
        단속장소.lat[j]=CCTV설치현황.lat[idx]

In [43]:
# 마지막 매핑작업
최종매칭전=단속장소[단속장소.lon.isna()]# 매핑 후, nan값
최종매칭전


for i in 최종매칭전[최종매칭전.re_loc.str.contains('대천초')].index:
    단속장소.lon[i],단속장소.lat[i]=keyword('대연동 대천초')
    
for i in 최종매칭전[최종매칭전.re_loc.str.contains('아이에스')].index:
    단속장소.lon[i],단속장소.lat[i]=keyword('용호동 동서아이에스')
    
for i in 최종매칭전[최종매칭전.re_loc.str.contains('푸르지오')].index:
    단속장소.lon[i],단속장소.lat[i]=keyword('대연동 푸르지오아파트')
    
for i in 최종매칭전[최종매칭전.re_loc.str.contains('경성대')].index:
    단속장소.lon[i],단속장소.lat[i]=keyword('대연동 경성대')  
for i in 최종매칭전[최종매칭전.re_loc.str.contains('김효준')].index:
    단속장소.lon[i],단속장소.lat[i]=keyword('대연동 김효준내과의원')
    
for i in 최종매칭전[최종매칭전.re_loc.str.contains('대교마트')].index:
    단속장소.lon[i],단속장소.lat[i]=keyword('대연동 대교마트')
    
for i in 최종매칭전[최종매칭전.re_loc.str.contains('용당요양병원')].index:
    단속장소.lon[i],단속장소.lat[i]=keyword('용당동 용당요양병원')

for i in 최종매칭전[최종매칭전.re_loc.str.contains('경희자강한의원')].index:
    단속장소.lon[i],단속장소.lat[i]=address('부산광역시 남구 용호로160번길 4')
for i in 최종매칭전[최종매칭전.re_loc.str.contains('환경공단')].index:
    단속장소.lon[i],단속장소.lat[i]=keyword('용호동 환경공단')

In [45]:
단속장소[단속장소.lon.isna()].info() # 798개

<class 'pandas.core.frame.DataFrame'>
Int64Index: 798 entries, 4890 to 9307
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   cd_loc  798 non-null    object 
 1   re_loc  798 non-null    object 
 2   count   798 non-null    int64  
 3   lon     0 non-null      float64
 4   lat     0 non-null      float64
dtypes: float64(2), int64(1), object(2)
memory usage: 37.4+ KB


In [58]:
# 좌표별 불법주정차 단속횟수 추출
# 최종적으로 5,745 건 
불법주정차위반위치 = 단속장소.groupby(['lon','lat'],as_index=False)['count'].sum().sort_values(by='count',ascending=False)

In [59]:
불법주정차위반위치

,lon,lat,count
3138,129.0906255622,35.1352555052,3830
3019,129.0899786927,35.1248844039,3479
493,129.0677799658,35.1369200688,3369
4475,129.1011880264,35.1333596545,3032
4863,129.1084764384,35.1260931548,2946
...,...,...,...
4036,129.0977053676,35.1254150209,1
749,129.0697901104,35.1355842795,1
4034,129.0976971773,35.1243345828,1
2413,129.0871834971,35.1346226795,1


## 4. 격자별 불법주정차 단속횟수 추출

In [57]:
# 격자별
def multipolygon_to_coordinates(x): 
    lon, lat = x[0].exterior.xy 
    return [[x, y] for x, y in zip(lon, lat)] 

def polygon_to_coordinates(x): 
    lon, lat = x.exterior.xy 
    return [[x, y] for x, y in zip(lon, lat)] 

In [62]:
import pyproj
from fiona.crs import from_epsg
import geopandas as gpd
불법주정차위경도= gpd.GeoDataFrame(불법주정차위반위치, geometry=gpd.points_from_xy(불법주정차위반위치.lon,불법주정차위반위치.lat))
gdf_불법주정차위경도 = gpd.GeoDataFrame(불법주정차위경도,crs=from_epsg(4326))

In [63]:
격자= gpd.read_file('15.부산시남구_격자(300X300).geojson') ## 파일 임포트 수정 필요
# geometry를 coordinate 형태로 적용
격자['coordinates'] = 격자['geometry'].apply(polygon_to_coordinates) #pydeck 을 위한 coordinate type
격자

,id,geometry,coordinates
0,6,"POLYGON ((129.06301 35.15031, 129.06570 35.150...","[[129.06300949513243, 35.15030633905416], [129..."
1,7,"POLYGON ((129.06301 35.14810, 129.06570 35.148...","[[129.06300949513243, 35.14810280145483], [129..."
2,8,"POLYGON ((129.06301 35.14590, 129.06570 35.145...","[[129.06300949513243, 35.14589920418718], [129..."
3,9,"POLYGON ((129.06301 35.14370, 129.06570 35.143...","[[129.06300949513243, 35.14369554725283], [129..."
4,10,"POLYGON ((129.06301 35.14149, 129.06570 35.141...","[[129.06300949513243, 35.14149183065344], [129..."
5,12,"POLYGON ((129.06301 35.13708, 129.06570 35.137...","[[129.06300949513243, 35.13708421846611], [129..."
6,13,"POLYGON ((129.06301 35.13488, 129.06570 35.134...","[[129.06300949513243, 35.134880322881465], [12..."
7,15,"POLYGON ((129.06301 35.13047, 129.06570 35.130...","[[129.06300949513243, 35.130472352738416], [12..."
8,18,"POLYGON ((129.06301 35.12386, 129.06570 35.123...","[[129.06300949513243, 35.1238599501142], [129...."
9,24,"POLYGON ((129.06301 35.11063, 129.06570 35.110...","[[129.06300949513243, 35.11063353435414], [129..."


In [66]:
# 가구수 좌표에 해당하는 격자표시
격자별_불법주정차=gpd.sjoin(격자, gdf_불법주정차위경도, how='left', op='intersects')

In [67]:
격자별_불법주정차

,id,geometry,coordinates,index_right,lon,lat,count
0,6,"POLYGON ((129.06301 35.15031, 129.06570 35.150...","[[129.06300949513243, 35.15030633905416], [129...",116.0000000000,129.0643214792,35.1481165323,1.0000000000
0,6,"POLYGON ((129.06301 35.15031, 129.06570 35.150...","[[129.06300949513243, 35.15030633905416], [129...",176.0000000000,129.0654385469,35.1482786218,9.0000000000
0,6,"POLYGON ((129.06301 35.15031, 129.06570 35.150...","[[129.06300949513243, 35.15030633905416], [129...",179.0000000000,129.0654693000,35.1482798997,284.0000000000
0,6,"POLYGON ((129.06301 35.15031, 129.06570 35.150...","[[129.06300949513243, 35.15030633905416], [129...",181.0000000000,129.0654737324,35.1482816265,34.0000000000
0,6,"POLYGON ((129.06301 35.15031, 129.06570 35.150...","[[129.06300949513243, 35.15030633905416], [129...",180.0000000000,129.0654725442,35.1482823676,3.0000000000
...,...,...,...,...,...,...,...
538,860,"POLYGON ((129.12769 35.11945, 129.13038 35.119...","[[129.12768819558906, 35.11945138344417], [129...",nan,nan,nan,nan
539,861,"POLYGON ((129.12769 35.11725, 129.13038 35.117...","[[129.12768819558906, 35.11724701063793], [129...",nan,nan,nan,nan
540,862,"POLYGON ((129.12769 35.11504, 129.13038 35.115...","[[129.12768819558906, 35.11504257818642], [129...",nan,nan,nan,nan
541,863,"POLYGON ((129.12769 35.11284, 129.13038 35.112...","[[129.12768819558906, 35.11283808609127], [129...",nan,nan,nan,nan


In [69]:
격자별_불법주정차.groupby('id',as_index=False)['count'].sum()

,id,count
0,6,336.0000000000
1,7,633.0000000000
2,8,620.0000000000
3,9,925.0000000000
4,10,169.0000000000
5,12,0.0000000000
6,13,0.0000000000
7,15,0.0000000000
8,18,0.0000000000
9,24,0.0000000000


In [70]:
# 격자별 가구수 합치기
격자['불법주정차']=격자별_불법주정차.groupby('id',as_index=False)['count'].sum()['count']

In [74]:
격자

,id,geometry,coordinates,불법주정차
0,6,"POLYGON ((129.06301 35.15031, 129.06570 35.150...","[[129.06300949513243, 35.15030633905416], [129...",336.0000000000
1,7,"POLYGON ((129.06301 35.14810, 129.06570 35.148...","[[129.06300949513243, 35.14810280145483], [129...",633.0000000000
2,8,"POLYGON ((129.06301 35.14590, 129.06570 35.145...","[[129.06300949513243, 35.14589920418718], [129...",620.0000000000
3,9,"POLYGON ((129.06301 35.14370, 129.06570 35.143...","[[129.06300949513243, 35.14369554725283], [129...",925.0000000000
4,10,"POLYGON ((129.06301 35.14149, 129.06570 35.141...","[[129.06300949513243, 35.14149183065344], [129...",169.0000000000
5,12,"POLYGON ((129.06301 35.13708, 129.06570 35.137...","[[129.06300949513243, 35.13708421846611], [129...",0.0000000000
6,13,"POLYGON ((129.06301 35.13488, 129.06570 35.134...","[[129.06300949513243, 35.134880322881465], [12...",0.0000000000
7,15,"POLYGON ((129.06301 35.13047, 129.06570 35.130...","[[129.06300949513243, 35.130472352738416], [12...",0.0000000000
8,18,"POLYGON ((129.06301 35.12386, 129.06570 35.123...","[[129.06300949513243, 35.1238599501142], [129....",0.0000000000
9,24,"POLYGON ((129.06301 35.11063, 129.06570 35.110...","[[129.06300949513243, 35.11063353435414], [129...",0.0000000000


In [76]:
# 정규화 # StandardScaler() --> MinMaxScaler()

# 패키지 호출
from sklearn.preprocessing import MinMaxScaler
# 객체 생성 
scaler = MinMaxScaler()
# 모델 fit
x=격자['불법주정차'].to_numpy() # Series.to_numpy()
x= x.reshape(-1,1)
scaled = scaler.fit_transform(x)


In [77]:
격자['정규화_불법주정차']=scaled
격자

,id,geometry,coordinates,불법주정차,정규화_불법주정차
0,6,"POLYGON ((129.06301 35.15031, 129.06570 35.150...","[[129.06300949513243, 35.15030633905416], [129...",336.0000000000,0.0478973628
1,7,"POLYGON ((129.06301 35.14810, 129.06570 35.148...","[[129.06300949513243, 35.14810280145483], [129...",633.0000000000,0.0902352103
2,8,"POLYGON ((129.06301 35.14590, 129.06570 35.145...","[[129.06300949513243, 35.14589920418718], [129...",620.0000000000,0.0883820385
3,9,"POLYGON ((129.06301 35.14370, 129.06570 35.143...","[[129.06300949513243, 35.14369554725283], [129...",925.0000000000,0.1318602994
4,10,"POLYGON ((129.06301 35.14149, 129.06570 35.141...","[[129.06300949513243, 35.14149183065344], [129...",169.0000000000,0.0240912331
5,12,"POLYGON ((129.06301 35.13708, 129.06570 35.137...","[[129.06300949513243, 35.13708421846611], [129...",0.0000000000,0.0000000000
6,13,"POLYGON ((129.06301 35.13488, 129.06570 35.134...","[[129.06300949513243, 35.134880322881465], [12...",0.0000000000,0.0000000000
7,15,"POLYGON ((129.06301 35.13047, 129.06570 35.130...","[[129.06300949513243, 35.130472352738416], [12...",0.0000000000,0.0000000000
8,18,"POLYGON ((129.06301 35.12386, 129.06570 35.123...","[[129.06300949513243, 35.1238599501142], [129....",0.0000000000,0.0000000000
9,24,"POLYGON ((129.06301 35.11063, 129.06570 35.110...","[[129.06300949513243, 35.11063353435414], [129...",0.0000000000,0.0000000000


In [15]:
## 불법주정차_격자별_매핑_완료.csv 로 저장.
격자.to_csv('불법주정차_격자별_매핑_완료.csv')